In [1]:
import sqlite3
import pandas as pd
import psycopg2

In [2]:
pwd

'/Users/amlvt225/Code/GitHub/donkey_golf/jupyter_notebooks'

In [3]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [4]:
ls

README.md          jupyter_notebooks/ run.py
donkey_golf/       requirements.txt


In [5]:
from donkey_golf import data_utils as du

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
/Users/amlvt225/Code/GitHub/donkey_golf/donkey_golf/data_utils.py:28: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_sql_dict = oyaml.load(open(conf.yaml_sql_loc))


In [19]:
du.run_sql('SELECT * FROM teams LIMIT 5')

,id,tourney_id,golfer
0,4,2748486273828282,Justin Rose
1,4,2748486273828282,Dustin Johnson
2,4,2748486273828282,Rory McIlroy
3,4,2748486273828282,Takumi Kanaya (a)
4,4,2748486273828282,Si Woo Kim


In [20]:
du.run_sql('SELECT * FROM user LIMIT 5')

,user
0,masjfonv


In [21]:
du.run_sql('SELECT * FROM leaderboard LIMIT 5')

,pos,unnamed:_1,player,to_par,today,thru,r1,r2,r3,r4,total_strokes,tourney_id,donkey_score,missed_cut,player_left
0,1,-,Shane Lowry,-8,-2,2,65,--,--,--,65,401056527,-8,0,1
1,T2,13,Emiliano Grillo,-7,-4,F,68,67,--,--,135,401056527,-7,0,1
2,T2,13,Dustin Johnson,-7,-4,F,68,67,--,--,135,401056527,-7,0,1
3,T4,3,Sam Burns,-5,-1,F,67,70,--,--,137,401056527,-5,0,1
4,T4,11,Charley Hoffman,-5,-2,F,68,69,--,--,137,401056527,-5,0,1


In [ ]:
sql = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname='public';"

In [ ]:
du.run_sql(sql)

In [ ]:
conf = du.config.DGConfig()

In [ ]:
df.to_sql()

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine(conf.db_url)

In [ ]:
with engine.connect() as conn:
    print(conn)
    df.to_sql('WOOFREAKINGWHOO', conn, if_exists='replace', index=False)

In [ ]:
with engine.connect() as conn:
    print(conn)
    print(pd.read_sql('SELECT * FROM teams', conn))

In [ ]:
with psycopg2.connect(host=conf.db_host,
                        database=conf.db_database,
                        user=conf.db_user,
                        password=conf.db_key) as conn:
    
        df.to_sql('WOOFREAKINGWHOO', conn, if_exists='replace', index=False)

In [ ]:
df = du.run_sql('SELECT * FROM teams')

In [ ]:
df

In [ ]:
du.data_load_leaderboard()

In [ ]:
du.pull_tourney_leaderboard()

In [ ]:
du.pull_available_players()

In [ ]:
du.calculate_cut_line()

In [ ]:
df = du.login_checker('po@poo.com')

In [ ]:
df

In [ ]:
df['password'][0]

In [ ]:
df

In [ ]:
if df.empty:
    print('yay')
else:
    print('match')

# Show all tables in our database 

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
user_id=6
sql = f"""
SELECT * from teams a
INNER JOIN leaderboard b
    ON a.golfer = b.player
WHERE id = {user_id}    
"""
pd.read_sql(sql, conn)

In [ ]:
pd.read_sql('SELECT * from leaderboard', conn)

In [ ]:
pd.read_sql('SELECT * from user', conn)